### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[  7.  10.   8.]
 [  6.  10.   7.]
 [  8.   9.   8.]
 [  8.  10.  10.]
 [  9.   9.   9.]
 [  8.   9.   9.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the funtion on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of matrix factorization.

In [20]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of users in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat)) # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://numpy.org/doc/stable/reference/random/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features)# user matrix filled with random values of shape user x latent
    movie_mat = np.random.rand(latent_features, n_movies)# movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # keep track of iteration and MSE
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):
        
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff ** 2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2 * diff * movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2 * diff * user_mat[i, k])

        # print results
        print(f'{iteration + 1}            {sse_accum/num_ratings}')
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 3 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [21]:
# use your function with 3 latent features, lr of 0.005 and 10 iterations
user_mat, movie_mat = FunkSVD(ratings_mat,
                             latent_features = 3,
                             learning_rate = 0.005,
                             iters = 10)

Optimization Statistics
Iterations | Mean Squared Error 
1            59.42171938739015
2            46.53876808777917
3            30.7231315514702
4            16.228753466880594
5            6.932068189215632
6            2.718436811784318
7            1.2432426661276128
8            0.7831281266485681
9            0.6339396196727126
10            0.5774650951372735


In [22]:
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[  7.21571271   8.82802867   8.3313226 ]
 [  6.92222812   8.56025915   7.71277214]
 [  7.51858046   9.18232511   8.37877968]
 [  8.32965258  10.35993024   9.37641966]
 [  8.06155701   9.76132781   8.82103921]
 [  7.67910869   9.45572721   8.57724393]]
[[ nan  10.   8.]
 [  6.  10.   7.]
 [  8.   9.   8.]
 [  8.  10.  10.]
 [  9.   9.   9.]
 [  8.   9.   9.]]


_**Task: compare the predicted vs actual ratings and write your findings here**_

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 3 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 300.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about the `mean squared error` at the end of each training iteration?

In [23]:
#use your function with 3 latent features, lr of 0.005 and 300 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, 
                              latent_features=3, 
                              learning_rate=0.005, 
                              iters=300)

Optimization Statistics
Iterations | Mean Squared Error 
1            65.66544623200366
2            56.494860675492085
3            42.96953214017161
4            27.048012784408673
5            13.398544158620666
6            5.285998649594123
7            1.8754127196813404
8            0.7680294192195053
9            0.45250066220131774
10            0.3608766995832256
11            0.32851794860713224
12            0.31243701754808634
13            0.3013785942193839
14            0.2921430471100117
15            0.2836942557320701
16            0.27564916867361394
17            0.2678521414050801
18            0.2602375733370994
19            0.2527789769945574
20            0.24546786038339952
21            0.238304257424462
22            0.23129224373515275
23            0.22443772019269662
24            0.21774727207755248
25            0.21122755525262038
26            0.20488494874396948
27            0.1987253469796096
28            0.19275402893291926
29            0.18697

In [24]:
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[  6.87660523  10.00599482   7.99424022]
 [  5.85119684   9.93930557   7.16065878]
 [  7.6455348    8.85918797   8.38078271]
 [  8.53261893  10.21341395   9.43690926]
 [  8.73685867   8.89325386   9.27946329]
 [  8.13055431   9.05230993   8.86369852]]
[[ nan  10.   8.]
 [  6.  10.   7.]
 [  8.   9.   8.]
 [  8.  10.  10.]
 [  9.   9.   9.]
 [  8.   9.   9.]]


_**Task: compare the predicted vs actual ratings and write your findings here**_

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 3 latent features, a learning rate of 0.005, and 450 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [17]:
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,   8.],
        [  6.,  10.,   7.],
        [  8.,   9.,   8.],
        [  8.,  10.,  10.],
        [  9.,   9.,   9.],
        [  8.,   9.,   9.]])

In [25]:
# run SVD on the matrix with the missing value
#use your function with 3 latent features, lr of 0.005 and 450 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, 
                              latent_features=3, 
                              learning_rate=0.005, 
                              iters=450)

Optimization Statistics
Iterations | Mean Squared Error 
1            59.48967360684325
2            46.348173161910935
3            30.454752824996966
4            16.002427955721576
5            6.779551669871756
6            2.6159414469738236
7            1.1648353906638729
8            0.7156083346382847
9            0.5717673512096598
10            0.5181310054780441
11            0.4937705176897045
12            0.4805203967113677
13            0.47214528071867484
14            0.4661272148973155
15            0.4613077059366519
16            0.4571050979776096
17            0.453210219665377
18            0.44945201399399853
19            0.4457328604294967
20            0.4419955911295565
21            0.43820596017600527
22            0.4343430256218196
23            0.43039373898729644
24            0.42634984178878443
25            0.422206059582542
26            0.41795904097508946
27            0.41360673138985066
28            0.4091480037153654
29            0.404582442

In [26]:
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is: {}".format(preds[0,0]))
print()
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 7. Not bad!")

The predicted value for the missing rating is: 7.9410753107995085

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 7. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 3 latent features, 0.005 learning rate, and 500 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [ ]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
#fit to 1000 users with 3 latent features, lr of 0.005, and 500 iterations
user_mat, movie_mat = FunkSVD(first_1000_users, 
                              latent_features=3, 
                              learning_rate=0.005, 
                              iters=500)

Optimization Statistics
Iterations | Mean Squared Error 
1            22.948422437927466
2            10.704126896983645
3            7.383590993918369
4            5.719147803215943
5            4.698906495627707
6            4.0040852325921215
7            3.4987091123634646
8            3.11350292108782
9            2.808997728074973
10            2.561206938176742
11            2.3549163162181244
12            2.1801285746907295
13            2.030047192789999
14            1.8998940339234183
15            1.786199066450149
16            1.686361375273596
17            1.598368462424524
18            1.5206112196629327
19            1.4517604933731862
20            1.39068606533165
21            1.3364057025252862
22            1.2880548803938894
23            1.2448695253980306
24            1.2061758421077249
25            1.1713831648457929
26            1.1399775320799812
27            1.111515022887243
28            1.0856147018970064
29            1.0619513732850774
30       

`6.` Now that you have a set of predictions for each user-movie pair.  Let's answer a few questions about your results. Provide the correct values to each of the variables below, and check your solutions using the tests below.

In [ ]:
# How many actual ratings exist in first_1000_users
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't have ratings
ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

In [ ]:
# Test your results against the solution
assert num_ratings == 14287, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 13835713, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")